In [5]:
import math
import numpy as np
import cv2
import PIL
import torch
from PIL import Image,ImageFilter
from torch.autograd import Variable
import torch.utils.data as data
import os
import os.path

# 生成卷积核和锚点
def motion_kernel(length, angle):
    EPS = np.finfo(float).eps
    alpha = (angle - math.floor(angle / 180) * 180) / 180 * math.pi
    half = length / 2
    cosalpha = math.cos(alpha)
    sinalpha = math.sin(alpha)
    if cosalpha < 0:
        xsign = -1
    elif angle == 90:
        xsign = 0
    else:
        xsign = 1
    psfwdt = 1;
    # 模糊核大小
    sx = int(math.fabs(length * cosalpha + psfwdt * xsign - length * EPS))
    sy = int(math.fabs(length * sinalpha + psfwdt - length * EPS))
    psf1 = np.zeros((sy, sx))

    # psf1是左上角的权值较大，越往右下角权值越小的核。
    # 这时运动像是从右下角到左上角移动
    for i in range(0, sy):
        for j in range(0, sx):
            psf1[i][j] = i * math.fabs(cosalpha) - j * sinalpha
            rad = math.sqrt(i * i + j * j)
            if rad >= half and math.fabs(psf1[i][j]) <= psfwdt:
                temp = half - math.fabs((j + psf1[i][j] * sinalpha) / cosalpha)
                psf1[i][j] = math.sqrt(psf1[i][j] * psf1[i][j] + temp * temp)
            psf1[i][j] = psfwdt + EPS - math.fabs(psf1[i][j]);
            if psf1[i][j] < 0:
                psf1[i][j] = 0
    # 运动方向是往左上运动，锚点在（0，0）
    anchor = (0, 0)
    # 运动方向是往右上角移动，锚点一个在右上角
    # 同时，左右翻转核函数，使得越靠近锚点，权值越大
    if angle < 90 and angle > 0:
        psf1 = np.fliplr(psf1)
        anchor = (psf1.shape[1] - 1, 0)
    elif angle > -90 and angle < 0:  # 同理：往右下角移动
        psf1 = np.flipud(psf1)
        psf1 = np.fliplr(psf1)
        anchor = (psf1.shape[1] - 1, psf1.shape[0] - 1)

    elif anchor < -90:  # 同理：往左下角移动
        psf1 = np.flipud(psf1)
        anchor = (0, psf1.shape[0] - 1)

    kernel = psf1 / psf1.sum()
    return kernel, anchor

def motion_blur(img,kernel, anchor):
    motion_blur = cv2.filter2D(np.array(img), -1, kernel, anchor=anchor)
    return PIL.Image.fromarray(motion_blur)

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP','.tif',
]
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def HR2LR(img,motion_kernel=None,motion_anchor=None,gauss=None,scale=None):
    if motion_kernel is not None and motion_anchor is not None:
        img = motion_blur(img,motion_kernel,motion_anchor)
    if gauss is not None:
        img = img.filter(ImageFilter.GaussianBlur(gauss))
    if scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image.ANTIALIAS)
    return img

In [6]:
images = []
motion_len = 10
motion_angel = 66
gauss = 5
kernel,anchor = motion_kernel(motion_len, motion_angel)
imgdir = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/images/style-images"
lrimgdir = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test"
if not os.path.exists(lrimgdir):
    os.mkdir(lrimgdir)
for img in os.listdir(imgdir):
    hrname = os.path.join(imgdir, img)
    lrsavename = os.path.join(lrimgdir, img)
    if is_image_file(img):
        images.append(img)
        hr = Image.open(hrname)
        lr = HR2LR(hr,kernel, anchor,gauss)
        lr.save(lrsavename)
        print("save: "+lrsavename)


save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/foreman.tif
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/comic.bmp
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/starfish.tif
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/timg.jpg
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/butterfly.tif
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/pepper.bmp
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/man.bmp
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/girl.tif
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/flowers.bmp
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/lena.tif
save: /home/wcd/Projects/Pytorch-examples/fast_neural_style/images/test/zebra.bmp


In [31]:
import torch


class LRNet(torch.nn.Module):
    def __init__(self):
        super(LRNet, self).__init__()
        # Initial convolution layers
        self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(32, affine=True)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = torch.nn.InstanceNorm2d(64, affine=True)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = torch.nn.InstanceNorm2d(128, affine=True)
        # Residual layers
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)
        # Conv+PS upsampling Layers
        self.ps1 = ConvPSLayer(128, 32, kernel_size=3, stride=1, upsample=2)
        self.in4 = torch.nn.InstanceNorm2d(32, affine=True)
        self.ps2 = ConvPSLayer(32, 8, kernel_size=3, stride=1, upsample=2)
        self.in5 = torch.nn.InstanceNorm2d(8, affine=True)
        self.conv4 = ConvLayer(8, 3, kernel_size=1, stride=1)
        # Non-linearities
        self.relu = torch.nn.ReLU()


    def forward(self, X):
        y = self.relu(self.in1(self.conv1(X)))
        y = self.relu(self.in2(self.conv2(y)))
        y = self.relu(self.in3(self.conv3(y)))
        y = self.res1(y)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.relu(self.in4(self.ps1(y)))
        y = self.relu(self.in5(self.ps2(y)))
        y = self.conv4(y)
        return y
    

class ConvLayer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)


    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out


class ResidualBlock(torch.nn.Module):
    """ResidualBlock
    introduced in: https://arxiv.org/abs/1512.03385
    recommended architecture: http://torch.ch/blog/2016/02/04/resnets.html
    """

    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out


class UpsampleConvLayer(torch.nn.Module):
    """UpsampleConvLayer
    Upsamples the input and then does a convolution. This method gives better results
    compared to ConvTranspose2d.
    ref: http://distill.pub/2016/deconv-checkerboard/
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        if upsample:
            self.upsample_layer = torch.nn.UpsamplingNearest2d(scale_factor=upsample)
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = self.upsample_layer(x_in)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out


class ConvPSLayer(torch.nn.Module):
    """
    conv of (in,outxuxu,k,s)
    follow by
    pixel shuffle of scalling factor (u)
    output is (bs out hxu wxu)
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(ConvPSLayer, self).__init__()
        self.upsample = upsample
        if upsample:
            self.pixelshuffle_layer = torch.nn.PixelShuffle(upsample)
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels*upsample^2, kernel_size, stride)

    def forward(self, x):

        out = self.reflection_pad(x)
        out = self.conv2d(out)
        if self.upsample:
            out = self.pixelshuffle_layer(out)
        return out


In [60]:
lrnet = LRNet()
import torch.nn.init as init
lrnetchild = lrnet.modules()
for i in lrnetchild:
    classname = i.__class__.__name__
    print(classname)
    if classname.find('Conv') != -1:
        i.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        i.weight.data.normal_(1.0, 0.02)
        i.bias.data.fill_(0)

LRNet
ConvLayer


AttributeError: 'ConvLayer' object has no attribute 'weight'

In [46]:
init.xavier_normal(lrnet.parameters())

AttributeError: 'generator' object has no attribute 'ndimension'

In [66]:
for name, param in lrnet.named_parameters():
    print(name)
    print(param.size)

conv1.conv2d.weight
<built-in method size of torch.FloatTensor object at 0x7fbb937a6fc8>
conv1.conv2d.bias
<built-in method size of torch.FloatTensor object at 0x7fbb937a6788>
in1.weight
<built-in method size of torch.FloatTensor object at 0x7fbb937a6a88>
in1.bias
<built-in method size of torch.FloatTensor object at 0x7fbb937a6908>
conv2.conv2d.weight
<built-in method size of torch.FloatTensor object at 0x7fbb937a6108>
conv2.conv2d.bias
<built-in method size of torch.FloatTensor object at 0x7fbb93776d08>
in2.weight
<built-in method size of torch.FloatTensor object at 0x7fbb93776c88>
in2.bias
<built-in method size of torch.FloatTensor object at 0x7fbb93776c48>
conv3.conv2d.weight
<built-in method size of torch.FloatTensor object at 0x7fbb937b6288>
conv3.conv2d.bias
<built-in method size of torch.FloatTensor object at 0x7fbb937b6648>
in3.weight
<built-in method size of torch.FloatTensor object at 0x7fbb937b6448>
in3.bias
<built-in method size of torch.FloatTensor object at 0x7fbb937b6608

In [67]:
lrnet.__class__.__name__

'LRNet'

In [76]:
   
classname = LRNet().__class__.__name__
print(classname)
print(classname.find('LRNet'))

LRNet
0


In [81]:
import torch
from PIL import Image,ImageFilter
from torch.autograd import Variable
import torch.utils.data as data
import os
import os.path
import filters

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

#load img from floder using filename
def load_HR_image(filename, size=None, mode='Y'):
    if mode is "Y":
        y, cb, cr = Image.open(filename).convert('L').split()
        img = y
    elif mode is 'RGB':
        img = Image.open(filename).convert('RGB')
    elif mode is 'YCbCr':
        img = Image.open(filename).convert('YCbCr')
    if size is not None:
        img = img.resize((size, size), Image.ANTIALIAS)
    return img

def load_LR_image(filename, motion_len=0,motion_angel=0,gauss=0,size=None,scale=None, mode='Y'):
    if mode is "Y":
        y, cb, cr = Image.open(filename).convert('L').split()
        img = y
    elif mode is 'RGB':
        img = Image.open(filename).convert('RGB')
    elif mode is 'YCbCr':
        img = Image.open(filename).convert('YCbCr')
    if size is not None:
        img = img.resize((size, size), Image.ANTIALIAS)
    elif scale is not None:
        img = img.resize((int(size / scale), int(size / scale)), Image.ANTIALIAS)
    if motion_len is not 0 and motion_angel is not 0:
        motion_kernel,motion_anchor = filters.motion_kernel(motion_len,motion_angel)
        img = filters.motion_blur(img,motion_kernel,motion_anchor)
    if gauss is not 0:
        img = img.filter(ImageFilter.GaussianBlur(gauss))

    return img

#Load content img
def default_loader(path):
    l = Image.open(path).convert('L')
    return Image.merge("RGB", (l, l, l))

#transfer HR PIL img to LR PIL img
def HR2LR(img,motion_kernel=None,motion_anchor=None,gauss=None,scale=None):
    if motion_kernel is not None and motion_anchor is not None:
        img = filters.motion_blur(img,motion_kernel,motion_anchor)
    if gauss is not None:
        img = img.filter(ImageFilter.GaussianBlur(gauss))
    if scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image.ANTIALIAS)
    return img

class trainingFolder(data.Dataset):

    def __init__(self, root, HR_size, LR_scale=None,
                 motion_len=None,motion_angel=None,gauss=None,
                 transform=None, target_transform=None,loader=load_HR_image,mode='Y'):
        super(trainingFolder,self).__init__()
        self.loader = loader
        if motion_len is not None and motion_angel is not None:
            self.motion_kernel, self.motion_anchor = filters.motion_kernel(motion_len, motion_angel)
        self.gauss = gauss
        self.HR_size = HR_size
        self.LR_scale = LR_scale
        self.mode=mode
        self.transform = transform
        self.target_transform = target_transform
        self.imgs = make_dataset(root)
        if len(self.imgs ) == 0:
            raise (RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                                                                             "Supported image extensions are: " + ",".join(
                IMG_EXTENSIONS)))
    def __getitem__(self, index):
        path = self.imgs[index]
        #training img--content
        img = self.loader(path,self.HR_size,mode=self.mode)
        #target img--reference
        target = HR2LR(img,self.motion_kernel,self.motion_anchor,self.gauss,self.LR_scale)

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def __len__(self):
        return len(self.imgs)


def make_dataset(dir):
    images = []
    for target in os.listdir(dir):
        fname = os.path.join(dir, target)
        if is_image_file(fname):
            images.append(fname)
    return images

def save_image(filename, data):
    img = data.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype("uint8")
    img = Image.fromarray(img)
    img.save(filename)


def gram_matrix(y):
    (b, ch, h, w) = y.size()
    features = y.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t) / (ch * h * w)
    return gram


def normalize_batch(batch):
    # normalize using imagenet mean and std
    mean = batch.data.new(batch.data.size())
    std = batch.data.new(batch.data.size())
    mean[:, 0, :, :] = 0.485
    mean[:, 1, :, :] = 0.456
    mean[:, 2, :, :] = 0.406
    std[:, 0, :, :] = 0.229
    std[:, 1, :, :] = 0.224
    std[:, 2, :, :] = 0.225
    batch = torch.div(batch, 255.0)
    batch -= Variable(mean)
    batch /= Variable(std)
    return batch


ModuleNotFoundError: No module named 'filters'

In [80]:
import argparse
import os
import sys
import time
from PIL import Image
import numpy as np
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

import nerual_style
from LRnet import TransformerNet
import visdom

#learn LR ,but reference is itself
def check_paths(args):
    try:
        if not os.path.exists(args.save_model_dir):
            os.makedirs(args.save_model_dir)
        if args.checkpoint_model_dir is not None and not (os.path.exists(args.checkpoint_model_dir)):
            os.makedirs(args.checkpoint_model_dir)
    except OSError as e:
        print(e)
        sys.exit(1)



args.subcommand = "train"
args.cuda = 1
args.save_model_dir = "ckpt"
args.dataset = '/home/wcd/LinkToMyLib/Datas/train2014'
args.style_image = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/images/style-images/comic.bmp"
args.epochs = 2
args.batch_size = 8
args.lr = 1e-3
args.content_weight = 1e3
args.style_weight = 1e10
args.checkpoint_model_dir = "ckpt"
args.checkpoint_interval = 2000
args.log_interval = 200/args.batch_size
args.plot_interval = 64
args.seed = 42
args.image_size = 128
args.style_size = None
args.HR_size = 128
args.LR_scale = 1
args.motion_len = 10
args.motion_angel = 66
args.gauss = 5
mode = args.mode = 'YCbCr'
#args.subcommand = "eval"
args.model = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/neural_style/ckpt/ckpt_epoch_0_batch_id_6000.pth"
args.content_image = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/images/style-images/flowers.bmp"
args.content_scale = 2
args.output_image = "/home/wcd/Projects/Pytorch-examples/fast_neural_style/images/output-images/flowers-6000.jpg"
torch.backends.cudnn.benchmark = True




ModuleNotFoundError: No module named 'nerual_style'